# client-server

In [1]:
import multiprocessing as mp
import socket
import time

In [2]:
mp.get_all_start_methods(), mp.get_start_method()

(['spawn', 'fork', 'forkserver'], 'spawn')

In [3]:
ctx = mp.get_context("fork")

In [5]:
def client(data):
    print("start client")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect(("localhost", 20_000))
        sock.sendall(data.encode())
        print("client sent data", data)

        data = sock.recv(1024)
        print("client recv data", data.decode())


def server():
    print("start server")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)

        sock.bind(("localhost", 20_000))
        sock.listen(5)

        i = 0
        while i < 5:
            i += 1

            client_sock, addr = sock.accept()
            print("accept client", addr, client_sock)
    
            with client_sock:
                data = client_sock.recv(1024)
                data = data.decode()
                print("server recv data", data)
                client_sock.sendall(data.upper().encode())
                print("server sent all!")


client_proc = ctx.Process(target=client, args=("властелин колец",))
server_proc = ctx.Process(target=server)

server_proc.start()
time.sleep(1)
client_proc.start()

client_proc.join()
# server_proc.join()

start server
start client
client sent dataaccept client  властелин колец('127.0.0.1', 60509)
 <socket.socket fd=45, family=2, type=1, proto=0, laddr=('127.0.0.1', 20000), raddr=('127.0.0.1', 60509)>
server recv data властелин колец
server sent all!client recv data 
ВЛАСТЕЛИН КОЛЕЦ
accept client ('127.0.0.1', 60551) <socket.socket fd=45, family=2, type=1, proto=0, laddr=('127.0.0.1', 20000), raddr=('127.0.0.1', 60551)>
server recv data матрица
server sent all!
accept client ('127.0.0.1', 60584) <socket.socket fd=45, family=2, type=1, proto=0, laddr=('127.0.0.1', 20000), raddr=('127.0.0.1', 60584)>
server recv data матрица
server sent all!
accept client ('127.0.0.1', 60610) <socket.socket fd=45, family=2, type=1, proto=0, laddr=('127.0.0.1', 20000), raddr=('127.0.0.1', 60610)>
server recv data матрица
server sent all!
accept client ('127.0.0.1', 60655) <socket.socket fd=45, family=2, type=1, proto=0, laddr=('127.0.0.1', 20000), raddr=('127.0.0.1', 60655)>
server recv data last of us
serv

In [6]:
client_proc = ctx.Process(target=client, args=("матрица",))
client_proc.start()
client_proc.join()

start client
client sent data матрица
client recv data МАТРИЦА


In [7]:
client_proc = ctx.Process(target=client, args=("матрица",))
client_proc.start()
client_proc.join()

start client
client sent data матрица
client recv data МАТРИЦА


In [8]:
client_proc = ctx.Process(target=client, args=("матрица",))
client_proc.start()
client_proc.join()

start client
client sent data матрица
client recv data МАТРИЦА


In [9]:
client_proc = ctx.Process(target=client, args=("last of us",))
client_proc.start()
client_proc.join()

start client
client sent data last of us
client recv data LAST OF US


In [ ]:
client_proc = ctx.Process(target=client, args=("last of us",))
client_proc.start()
client_proc.join()

In [10]:
client_proc = ctx.Process(target=client, args=("last of us",))
client_proc.start()
client_proc.join()

start client


Process ForkProcess-9:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_21906/3857572068.py", line 4, in client
    sock.connect(("localhost", 20_000))
ConnectionRefusedError: [Errno 61] Connection refused


# generator-coroutine

In [12]:
def grep_gen(pattern):
    print("start grep for", pattern)
    i = 0
    while True:
        s = yield i
        i += 1
        if pattern in s:
            print("found!", s)
        else:
            print("no %s in %s" % (pattern, s))

In [13]:
grep = grep_gen("abc")

In [14]:
grep

<generator object grep_gen at 0x1096c56c0>

In [15]:
next(grep)

start grep for abc


0

In [16]:
grep.send("abc")

found! abc


1

In [17]:
grep.send("abcd")

found! abcd


2

In [18]:
grep.send("bcd")

no abc in bcd


3

In [19]:
next(grep)

TypeError: argument of type 'NoneType' is not iterable

In [20]:
grep.send("bcd")

StopIteration: 

In [21]:
grep = grep_gen("abcd")

In [22]:
grep.send("abdwkdj")

TypeError: can't send non-None value to a just-started generator

In [23]:
next(grep)

start grep for abcd


0

In [24]:
grep.send("abdwkdj")

no abcd in abdwkdj


1

In [25]:
grep = grep_gen("abcd")

In [26]:
grep.send(None)

start grep for abcd


0

In [27]:
grep.send("abdwkdj")

no abcd in abdwkdj


1

In [28]:
grep.throw(Exception("wrong"))

Exception: wrong

In [29]:
def counter(start, stop, step):
    while start < stop:
        step_cand = yield start
        
        if step_cand is not None:
            step = step_cand

        start += step


cnt = counter(0, 100, 1)
print("1:", next(cnt))
print("2:", next(cnt))

print("3:", cnt.send(10))
print("4:", next(cnt))

1: 0
2: 1
3: 11
4: 21


In [30]:
next(cnt)

31

In [31]:
cnt.send(4)

35

In [32]:
next(cnt)

39

In [33]:
next(cnt)

43

In [34]:
cnt.send(-10)

33

In [35]:
next(cnt)

23

In [36]:
next(cnt)

13

In [37]:
next(cnt)

3

In [38]:
next(cnt)

-7

In [39]:
cnt.send(30)

23

In [40]:
next(cnt)

53

In [41]:
next(cnt)

83

In [42]:
next(cnt)

StopIteration: 

In [43]:
import asyncio

In [44]:
#asyncio.run()

In [45]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, 'hello')
    await say_after(2, 'world')
    print(f"finished at {time.strftime('%X')}")

In [46]:
def syc_fn():
    await main()

SyntaxError: 'await' outside async function (1310051546.py, line 2)

In [49]:
t1 = time.time()

await main()

t2 = time.time()
print("tt", t2 - t1)

started at 20:37:24
hello
world
finished at 20:37:27
tt 3.0060980319976807


In [50]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    say1 = say_after(1, 'hello')
    await say_after(2, 'world')
    await say1
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("tt", t2 - t1)

started at 20:39:40
world
hello
finished at 20:39:43
tt 3.020059823989868


In [51]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    say1 = asyncio.create_task(say_after(1, 'hello'))
    await say_after(2, 'world')
    await say1
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("tt", t2 - t1)

started at 20:41:44
hello
world
finished at 20:41:46
tt 2.002563953399658


In [52]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    await asyncio.gather(
        say_after(1, 'hello'),
        say_after(2, 'world'),
    )
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("tt", t2 - t1)

started at 20:43:12
hello
world
finished at 20:43:14
tt 2.0017428398132324


In [56]:
async def countdown(n):
    for i in range(n):
        yield i
        print(f"yield {i=}")
        await asyncio.sleep(0)

In [57]:
async for x in countdown(5):
    print(x)

0
yield i=0
1
yield i=1
2
yield i=2
3
yield i=3
4
yield i=4


In [59]:
import aiohttp

In [64]:
URL = "https://docs.python.org/3/whatsnew/3.12.html"

async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            print(resp.status)
            data = await resp.text()
            return data


t1 = time.time()
data = await fetch_url(URL)
t2 = time.time()
print(t2 - t1)

200
0.3200948238372803


In [66]:
URL = "https://docs.python.org/3/whatsnew/3.12.html"
URLS = [URL] * 20

async def fetch_url(url, session):
    async with session.get(url) as resp:
        print(resp.status)
        data = await resp.text()
        return data


async def fetch_batch_urls(urls):
    async with aiohttp.ClientSession() as session:
        for url in urls:
            await fetch_url(url, session)


async def run():
    t1 = time.time()
    data = await fetch_batch_urls(URLS)
    t2 = time.time()
    print(t2 - t1)


await run()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
1.8855698108673096


In [74]:
URL = "https://docs.python.org/3/whatsnew/3.12.html"
URLS = [URL] * 40


async def fetch_url(url, session):
    async with session.get(url) as resp:
        # print(resp.status)
        data = await resp.text()
        return data


async def fetch_batch_urls(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            tasks.append(fetch_url(url, session))

        await asyncio.gather(*tasks)


async def run():
    t1 = time.time()
    data = await fetch_batch_urls(URLS)
    t2 = time.time()
    print(t2 - t1)


await run()

1.0576739311218262


In [84]:
URL = "https://docs.python.org/3/whatsnew/3.12.html"
URLS = [URL] * 5


async def fetch_url(url, session):
    async with session.get(url) as resp:
        # print(resp.status)
        data = await resp.text()
        return resp.status


async def fetch_batch_urls(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            tasks.append(asyncio.create_task(fetch_url(url, session)))

        for task in asyncio.as_completed(tasks):
            result = await task
            print("status", result)


async def run():
    t1 = time.time()
    data = await fetch_batch_urls(URLS)
    t2 = time.time()
    print(t2 - t1)


await run()

status 200
status 200
status 200
status 200
status 200
0.30745697021484375
